In [6]:
#@title Packages and functions
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import jv, jvp, yv, iv
from scipy.special import spherical_jn, spherical_yn
from scipy.optimize import root_scalar
plt.rcParams.update({'font.size': 22})
#%matplotlib qt

def Cooke_Rand_Modes(cL, cT, rho, eta, R, order, modes):
  def Mn_derivatives(n, A_n, R_n, bessel_type = 'jn'):
      '''First and second derivative of coefficient M_n
      according to Cooke and Rand (1973):
      Inputs:
      ------
      n = order number.
      A_n = coeffcient multiplying R_n.
      R_n = radius n.
      bessel_type = Either jn or yn for the spherical bessel function.
      Outputs:
      -------
      M_n = coeffcient M_n.
      M_np = first derivative of coeffcient M_n.
      M_npp = second derivative of coeffcient M_n.
      '''
      # First derivative of coeffcient M_n
      if bessel_type == 'jn':
          M_n = np.sqrt(2*np.pi/(A_n*R_n))*jv(n + 1/2, A_n*R_n)/2 #M_n = spherical_jn(n, A_n*R_n)
          M_np = np.sqrt(2*np.pi/(A_n*R_n))*(-A_n*R_n*jv(n + 3/2, A_n*R_n) + n*jv(n + 1/2, A_n*R_n))/(2*R_n) #M_np = spherical_jn(n, A_n*R_n, 1) # Eq-88
          # M_np = (n/(A_n*R_n))*spherical_jn(n, A_n*R_n) - spherical_jn(n + 1, A_n*R_n) # Eq-89
      elif bessel_type == 'yn':
          M_n = np.sqrt(2*np.pi/(A_n*R_n))*yv(n + 1/2, A_n*R_n)/2 #M_n = spherical_yn(n, A_n*R_n)
          M_np = A_n*np.sqrt(2*np.pi/(A_n*R_n))*(yv(n - 1/2, A_n*R_n)/2 - yv(n + 3/2, A_n*R_n)/2)/2 - np.sqrt(2*np.pi/(A_n*R_n))*yv(n + 1/2, A_n*R_n)/(4*R_n) #M_np = spherical_yn(n, A_n*R_n, 1) # Eq-88
          # M_np = (n/(A_n*R_n))*spherical_yn(n, A_n*R_n) - spherical_yn(n + 1, A_n*R_n) # Eq-89
      else:
          print('There is a typo on the bessel_type argument! Check it out!')
      # Second derivative of coeffcient M_n
      M_npp = -(2/R_n)*M_np - (A_n**2 - (n*(n + 1)/R_n**2))*M_n # Eq-90
      return M_n, M_np, M_npp


  def poisson_ratio(c_l, c_t):
      nu = (c_l**2 - 2*c_t**2)/(2*(c_l**2 - c_t**2))
      return nu

  def shear_modulus(c_t, rho):
      mu = rho*c_t**2
      return mu

  def Cooke_Rand_matrix(omega, kh, cL, cT, rho, eta, R):
      '''
      Determinant of coefficients for the 6x6 matrix in the paper:
      Cooke, J. R., & Rand, R. H. (1973). A mathematical study of resonance in intact fruits and vegetables using a 3-media elastic sphere model. Journal of agricultural engineering research, 18(2), 141-157.[doi: https://doi.org/10.1016/0021-8634(73)90023-1]
      kh = dimensionless wavenumber (equivalent to kp in Cerv's code)
      R_2 = Outer radius
      R_1 = Inner radius
      eta = R_1/R_2 (ratio of R_1 to R_2)
      cL1 = Longitudinal velocity of medium 1
      cT1 = Transversal velocity of medium 1
      cL2 = Longitudinal velocity of medium 2
      cT2 = Transversal velocity of medium 2
      '''
      cL1 = cL[0]
      cL2 = cL[1]
      cL3 = cL[2]
      cT1 = cT[0]
      cT2 = cT[1]
      cT3 = cT[2]
      rho1 = rho[0]
      rho2 = rho[1]
      rho3 = rho[2]
      # Calculate the mu coefficients
      mu1 = shear_modulus(cT1, rho1)
      mu2 = shear_modulus(cT2, rho2)
      mu3 = shear_modulus(cT3, rho3)
      nu1 = poisson_ratio(cL1, cT1)
      nu2 = poisson_ratio(cL2, cT2)
      nu3 = poisson_ratio(cL3, cT3)
      # re-scaling for numerical convenience
      # R = R#*1000
      # cL1 = cL1#/1000
      # cL2 = cL2#/1000
      # cL3 = cL3#/1000
      # cT1 = cT1#/1000
      # cT2 = cT2#/1000
      # cT3 = cT3#/1000
      # omega = omega#/1e6

      ##########################
      n = kh #- 0.5 #1
      #theta = 30*np.pi/180
      #eta = 0.3
      R_3 = R #1.00005*R_2
      R_2 = 0.99999999*R
      R_1 = eta*R_3
      # For medium 1
      A_1 = omega/cL1
      A_2 = omega/cT1
      #eta = np.cos(theta)
      # For medium 2
      A_3 = omega/cL2
      A_4 = omega/cT2

      #spherical_jn(n, z[, derivative])
      # bessel function constants
      M_1, M_1p, M_1pp = Mn_derivatives(n, A_1, R_1, bessel_type = 'jn')
      M_2, M_2p, M_2pp = Mn_derivatives(n, A_2, R_1, bessel_type = 'jn')
      M_3, M_3p, M_3pp = Mn_derivatives(n, A_3, R_1, bessel_type = 'jn')
      M_4, M_4p, M_4pp = Mn_derivatives(n, A_4, R_1, bessel_type = 'jn')
      M_5, M_5p, M_5pp = Mn_derivatives(n, A_3, R_2, bessel_type = 'jn')
      M_6, M_6p, M_6pp = Mn_derivatives(n, A_4, R_2, bessel_type = 'jn')
      M_7, M_7p, M_7pp = Mn_derivatives(n, A_3, R_1, bessel_type = 'yn')
      M_8, M_8p, M_8pp = Mn_derivatives(n, A_4, R_1, bessel_type = 'yn')
      M_9, M_9p, M_9pp = Mn_derivatives(n, A_3, R_2, bessel_type = 'yn')
      M_10, M_10p, M_10pp = Mn_derivatives(n, A_4, R_2, bessel_type = 'yn')

      # Components of the 6-by-6 determinant (Eqs.42-87)
      ###############################################################################
      S11 = M_1/(A_3*R_1)
      S12 = (M_2p/A_3) + (M_2/(A_3*R_1))
      S13 = -M_3/(A_3*R_1)
      S14 = -M_7/(A_3*R_1)
      S15 = (-M_4p/A_3) - (M_4/(A_3*R_1))
      S16 = (-M_8p/A_3) - M_8/(A_3*R_1)
      ###############################################################################
      S21 = M_1p/A_3
      S22 = n*(n + 1)*M_2/(A_3*R_1)
      S23 = -M_3p/A_3
      S24 = -M_7p/A_3
      S25 = -n*(n + 1)*M_4/(A_3*R_1)
      S26 = -n*(n + 1)*M_8/(A_3*R_1)
      # New constants H_1 and H_2
      H_1 = (nu1/(1 - 2*nu1))*(A_1/A_3)**2
      H_2 = nu2/(1 - 2*nu2)
      ###############################################################################
      S31 = -H_1*M_1 + M_1pp/(A_3**2)
      S32 = -n*(n + 1)*(M_2/(A_3*R_1)**2 - M_2p/(R_1*A_3**2))
      S33 = M_3*H_2 - M_3pp/(A_3**2)
      S34 = H_2*M_7 - M_7pp/(A_3**2)
      S35 = n*(n + 1)*(M_4/(A_3*R_1)**2 - M_4p/(R_1*A_3**2))
      S36 = n*(n + 1)*(M_8/(A_3*R_1)**2 - M_8p/(R_1*A_3**2))
      # New constant H_3
      H_3 = mu1/mu2
      ###############################################################################
      S41 = -2*H_3*(M_1/(A_3*R_1)**2 - M_1p/(R_1*A_3**2))
      S42 = H_3*(M_2pp/A_3**2 - 2*M_2/(A_3*R_1)**2 + n*(n + 1)*M_2/(A_3*R_1)**2)
      S43 = 2*(M_3/(A_3*R_1)**2 - M_3p/(R_1*A_3**2))
      S44 = 2*(M_7/(A_3*R_1)**2 - M_7p/(R_1*A_3**2))
      S45 = -M_4pp/A_3**2 - (M_4/(R_1*A_3**2)**2)*(n*(n + 1) - 2)
      S46 = -M_8pp/(A_3**2) + (M_8/(A_3*R_1)**2)*(n*(n + 1) - 2)
      ###############################################################################
      # New constants
      r_2 = R_2
      r_3 = R_3
      r_23 = r_2/r_3
      mu_23 = mu2/mu3
      rho_23 = rho2/rho3
      H_4 = 2.0 - ((A_4*R_2)**2)*(mu_23*(1 - nu3)/(2*rho_23*(1 + nu3)))
      H_5 = (A_3*R_2*mu_23*r_23*(1 - nu3))/((1 - r_23)*(1 + nu3))
      ###############################################################################
      S51 = 0.0
      S52 = 0.0
      S53 = (-n*(n + 1)*M_5/(A_3*R_2)) - H_2*H_5*M_5 + H_4*M_5p/A_3 \
          + H_5*M_5pp/(A_3**2)
      S54 = (-n*(n + 1)*M_9/(A_3*R_2)) - H_2*H_5*M_9 + H_4*M_9p/A_3 \
          + H_5*M_9pp/(A_3**2)
      S55 = (n*(n + 1)*M_6p/A_3)*(-1 + H_5/(A_3*R_2)) \
          + (n*(n + 1)*M_6/(A_3*R_2))*(-1 + H_4 - H_5/(A_3*R_2))
      S56 = (n*(n + 1)*M_10p/A_3)*(-1 + H_5/(A_3*R_2)) \
          + (n*(n + 1)*M_10/(A_3*R_2))*(-1 + H_4 - H_5/(A_3*R_2))
      ###############################################################################
      # New Constants
      H_6 = (A_3*R_2*mu_23*r_23*(1 - nu3))/(2*(1 - r_23))
      H_7 = (1 - nu3)*(1 + (mu_23*(A_4*R_2)**2)/(2*rho_23))
      H_8 = (1 + nu3) - H_6/(A_3*R_2)
      ###############################################################################
      S61 = 0.0
      S62 = 0.0
      S63 = -n*(n + 1)*M_5/(A_3*R_2) + H_7*M_5/(A_3*R_2) \
          + 2*H_6*M_5/(A_3*R_2)**2 + (H_8 - H_6/(A_3*R_2))*(M_5p/A_3)
      S64 = -n*(n + 1)*M_9/(A_3*R_2) + H_7*M_9/(A_3*R_2) \
          + 2*H_6*M_9/(A_3*R_2)**2 + (H_8 - H_6/(A_3*R_2))*(M_9p/A_3)
      S65 = (-n*(n + 1)*M_6/(A_3*R_2))*(1 - H_8) + H_7*M_6/(A_3*R_2) \
          + 2*H_6*M_6/(A_3*R_2)**2 + (-n*(n + 1) + H_7)*(M_6p/A_3) \
              - H_6*M_6pp/A_3**2
      S66 = (-n*(n + 1)*M_10/(A_3*R_2))*(1 - H_8) + H_7*M_10/(A_3*R_2) \
          + (-n*(n + 1) + H_7)*(M_10p/A_3) - H_6*M_10pp/A_3**2

      # Dispersion Matrix
      dij = np.array([
          [S11, S12, S13, S14, S15, S16 ],
          [S21, S22, S23, S24, S25, S26 ],
          [S31, S32, S33, S34, S35, S36 ],
          [S41, S42, S43, S44, S45, S46 ],
          [S51, S52, S53, S54, S55, S56 ],
          [S61, S62, S63, S64, S65, S66 ]
          ])

      return dij

  def D_det(omega):
      '''Calculate the determinant of matrix '''
      return np.linalg.det(Cooke_Rand_matrix(omega, kp, cL, cT, rho, eta, R))
    # Define constants and range of calculations
  fstop = 1.0*1e6 # Ending frequency in MHz
  dp = 1.0
  wstart = 0.0001*1e6;
  dw = 0.0001*1e6
  kpstart = 0
  kpstop = order # n values 3

  omegas = np.arange(wstart,2*np.pi*fstop,dw)
  kps = np.arange(kpstart,kpstop,dp)

  number_of_roots = modes # l values #10

  roots = []
  KP = []
  for kp in kps: #kp
      print(kp)
      print('mode', 'n', 'freq (Hz)', 'ka')
      rc = 0
      for omega in omegas:
          if kp < 1:
              NR = number_of_roots #- 1 #
              if rc < NR: #number_of_roots:
                  if (np.sign(D_det(omega)) > np.sign(D_det(omega+dw))) or (np.sign(D_det(omega)) < np.sign(D_det(omega+dw))):
                      #print(omega)\
                      root_val = root_scalar(D_det, bracket = [omega,omega+dw], method = 'bisect', xtol = 1e-7)# root_val.root
                      # if rc == 0:
                      #    roots.append(0)
                      #    KP.append(kp)
                      if root_val != None:
                          roots.append(root_val.root)
                          KP.append(kp)
                          print(''+str(int(kp))+'_S_'+str(rc)+'', kp, root_val.root/(2*np.pi), root_val.root*R/cT[1])
                          rc = rc + 1
                          # if rc == 1:
                          #     wstart = root_val.root
                          #     omegas = np.arange(wstart,2*np.pi*fstop,dw)
              elif rc == NR:
                  break
          elif kp >= 1:
              NR = number_of_roots #
              if rc < NR: #number_of_roots:
                  if (np.sign(D_det(omega)) > np.sign(D_det(omega+dw))) or (np.sign(D_det(omega)) < np.sign(D_det(omega+dw))):
                      #print(omega)\
                      root_val = root_scalar(D_det, bracket = [omega,omega+dw], method = 'bisect', xtol = 1e-7)# root_val.root
                      if root_val != None:
                          roots.append(root_val.root)
                          KP.append(kp)
                          print(''+str(int(kp))+'_S_'+str(rc)+'', kp, root_val.root/(2*np.pi), root_val.root*R/cT[1])
                          rc = rc + 1
                          if rc == 1:
                              wstart = root_val.root
                              omegas = np.arange(wstart,2*np.pi*fstop,dw)
              elif rc == NR:
                  break
  return roots, KP

# Calculate the fundamental eigenmodes of Cooke and Rand 1973

> Cooke, J. R., & Rand, R. H. (1973). A mathematical study of resonance in intact fruits and vegetables using a 3-media elastic sphere model. Journal of agricultural engineering research, 18(2), 141-157.[doi: https://doi.org/10.1016/0021-8634(73)90023-1]

In [7]:
#@title Calculate fundamental modes
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create input widgets
cL_widget = widgets.Text(value="1000, 1000, 1000", description="cL:")
cT_widget = widgets.Text(value="36, 36, 36", description="cT:")
rho_widget = widgets.Text(value="1010, 1010, 1010", description="rho:")
eta_widget = widgets.FloatSlider(value=1.0, min=0.01, max=1.0, step=0.01, description="eta:")
R_widget = widgets.FloatText(value=0.02627, description="R:")
order_widget = widgets.FloatText(value=3, description="order:")
modes_widget = widgets.FloatText(value=10, description="modes:")

# Create button to trigger calculation
button = widgets.Button(description="Calculate")

# Create output widget
output = widgets.Output()

# Define calculation function
def calculate(_):
    with output:
        clear_output()
        cL_values = [float(x) for x in cL_widget.value.split(",")]
        cT_values = [float(x) for x in cT_widget.value.split(",")]
        rho_values = [float(x) for x in rho_widget.value.split(",")]
        eta_value = eta_widget.value
        R_value = R_widget.value
        order_value = order_widget.value
        modes_value = modes_widget.value
        roots, KP = Cooke_Rand_Modes(cL_values, cT_values, rho_values, eta_value, R_value, order_value, modes_value)
    return roots, KP
# Attach calculation function to button click
button.on_click(calculate)

# Display widgets
display(order_widget, modes_widget, cL_widget, cT_widget, rho_widget, eta_widget, R_widget, button, output)

roots, KP = calculate(button)

FloatText(value=3.0, description='order:')

FloatText(value=10.0, description='modes:')

Text(value='1000, 1000, 1000', description='cL:')

Text(value='36, 36, 36', description='cT:')

Text(value='1010, 1010, 1010', description='rho:')

FloatSlider(value=1.0, description='eta:', max=1.0, min=0.01, step=0.01)

FloatText(value=0.02627, description='R:')

Button(description='Calculate', style=ButtonStyle())

Output()